In [1]:
import pandas as pd 
import numpy as np


In [2]:
horse_file = 'horse_data.csv'
full_df = pd.read_csv(horse_file, header=None, na_values='?')

In [3]:
# дам имена столбцам
columns = [
    "surgery", "age", "hospital_number", "rectal_temperature", "pulse", 
    "respiratory_rate", "temperature_of_extremities", "peripheral_pulse", 
    "mucous_membranes", "capillary_refill_time", "pain", "peristalsis", 
    "abdominal_distension", "nasogastric_tube", "nasogastric_reflux", 
    "nasogastric_reflux_ph", "rectal_examination_feces", "abdomen", 
    "packed_cell_volume", "total_protein", "abdominocentesis_appearance", 
    "abdomcentesis_total_protein", "outcome", "surgical_lesion", 
    "type_of_lesion_1", "type_of_lesion_2", "type_of_lesion_3", "cp_data"
]
full_df.columns = columns
df = (full_df[[
    'surgery', 'age','rectal_temperature', 'pulse', 
    'respiratory_rate', 'temperature_of_extremities', 'pain', 'outcome']].copy())
df


,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


# Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [4]:
df.columns

Index(['surgery', 'age', 'rectal_temperature', 'pulse', 'respiratory_rate',
       'temperature_of_extremities', 'pain', 'outcome'],
      dtype='object')

In [5]:
continuos_var = ['rectal_temperature', 'pulse', 'respiratory_rate']
categorical_var = ['surgery', 'age', 'temperature_of_extremities', 'pain', 'outcome']

### непрерывные переменные

In [6]:
df[continuos_var].describe()

,rectal_temperature,pulse,respiratory_rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


In [7]:
def remove_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_b = q1 - (1.5 * iqr)
    upper_b = q3 + (1.5 * iqr)
    return df[df[column].between(lower_b, upper_b, inclusive="both")]

remove_outliers_respiratory_rate = remove_outliers(df, 'respiratory_rate')
remove_outliers_rectal_temperature = remove_outliers(df, 'rectal_temperature')
remove_outliers_pulse = remove_outliers(df, 'pulse')


In [8]:
print(f"До удаления выбросов: {len(df)}")
print(f"После удаления выбросов (respiratory_rate): {len(remove_outliers_respiratory_rate)}")
print(f"После удаления выбросов (rectal_temperature): {len(remove_outliers_rectal_temperature)}")
print(f"После удаления выбросов (pulse): {len(remove_outliers_pulse)}")

До удаления выбросов: 300
После удаления выбросов (respiratory_rate): 225
После удаления выбросов (rectal_temperature): 226
После удаления выбросов (pulse): 271


In [9]:
print(remove_outliers_respiratory_rate['respiratory_rate'].mean(), 
     remove_outliers_respiratory_rate['respiratory_rate'].median(),
     remove_outliers_respiratory_rate['respiratory_rate'].mode()[0])
print("-------------------------")
print(df['respiratory_rate'].mean(), df['respiratory_rate'].median(), df['respiratory_rate'].mode()[0])

26.715555555555557 24.0 20.0
-------------------------
30.417355371900825 24.5 20.0


In [10]:
print(remove_outliers_rectal_temperature['rectal_temperature'].mean(), 
     remove_outliers_rectal_temperature['rectal_temperature'].median(),
     remove_outliers_rectal_temperature['rectal_temperature'].mode()[0])
print("-------------------------")
print(df['rectal_temperature'].mean(), df['rectal_temperature'].median(), df['rectal_temperature'].mode()[0])

38.16902654867257 38.2 38.0
-------------------------
38.16791666666667 38.2 38.0


In [11]:
print(remove_outliers_pulse['pulse'].mean(), 
     remove_outliers_pulse['pulse'].median(),
     remove_outliers_pulse['pulse'].mode()[0])
print("-------------------------")
print(df['pulse'].mean(), df['pulse'].median(), df['pulse'].mode()[0])

70.25830258302582 64.0 48.0
-------------------------
71.91304347826087 64.0 48.0


    мода не меняется. но я думаю что в непрерывных переменных она и не изменится если оствить или убрать выбросы.
    среднее меняется. Иногда даже сильно. Так что я думаю, что это указывает на влияние выбросов. 
    медиана не меняется или меняется на 0.5, для меня это поддтверждает ее устройчивость. поэтому буду заполянть потом пропуски в непрерывных переменных МЕДИАНОЙ.

### категориальные переменные

In [12]:
for col_name in categorical_var:
    print(f"столбец {col_name.upper()} {len(df[col_name])} {df[col_name].value_counts().sum()}")
    print(df[col_name].value_counts())
    print(f"УНИКАЛЬНЫЕ {df[col_name].unique()}")
    print(f"МОДА {df[col_name].mode()[0]}")
    print("*****************")

столбец SURGERY 300 299
surgery
1.0    180
2.0    119
Name: count, dtype: int64
УНИКАЛЬНЫЕ [ 2.  1. nan]
МОДА 1.0
*****************
столбец AGE 300 300
age
1    276
9     24
Name: count, dtype: int64
УНИКАЛЬНЫЕ [1 9]
МОДА 1
*****************
столбец TEMPERATURE_OF_EXTREMITIES 300 244
temperature_of_extremities
3.0    109
1.0     78
2.0     30
4.0     27
Name: count, dtype: int64
УНИКАЛЬНЫЕ [ 3. nan  1.  4.  2.]
МОДА 3.0
*****************
столбец PAIN 300 245
pain
3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: count, dtype: int64
УНИКАЛЬНЫЕ [ 5.  3.  2. nan  4.  1.]
МОДА 3.0
*****************
столбец OUTCOME 300 299
outcome
1.0    178
2.0     77
3.0     44
Name: count, dtype: int64
УНИКАЛЬНЫЕ [ 2.  3.  1. nan]
МОДА 1.0
*****************


In [13]:
df['age'] = df['age'].replace(9, 2)

In [14]:
df['age'].value_counts() # заменяю 9 на 2, так как в описании было сказано что есть только два варианта. 1 и 2

age
1    276
2     24
Name: count, dtype: int64

    по статистикам по категориальным переменным понятно что надо было заменить 9 на 2 в переменной age
    так же я вижу моду и уникальные значения, которые мне не дают какой то информации, кроме того что теперь я знаю какие значения ожидать в столбцах. И вижу как часто то или иное значение встречалось в данных

# Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.


In [15]:
df.groupby('surgery')[['pain', 'temperature_of_extremities']].value_counts()


surgery  pain  temperature_of_extremities
1.0      5.0   3.0                           19
         3.0   1.0                           16
         2.0   3.0                           13
         4.0   3.0                           12
         3.0   3.0                           12
         2.0   4.0                            7
               1.0                            7
         4.0   1.0                            7
         1.0   1.0                            5
         5.0   4.0                            5
               1.0                            5
         3.0   4.0                            4
         4.0   4.0                            3
         3.0   2.0                            3
         4.0   2.0                            2
         2.0   2.0                            2
         1.0   3.0                            2
               2.0                            1
         5.0   2.0                            1
2.0      1.0   1.0                           1

In [16]:
df.groupby('outcome')[['pain', 'temperature_of_extremities']].value_counts()

outcome  pain  temperature_of_extremities
1.0      3.0   1.0                           25
         1.0   1.0                           17
         2.0   1.0                           14
         3.0   3.0                           14
         1.0   2.0                           11
         2.0   3.0                            9
         5.0   3.0                            8
         3.0   2.0                            7
         4.0   3.0                            5
         1.0   3.0                            5
         3.0   4.0                            4
         4.0   2.0                            3
         2.0   4.0                            3
               2.0                            3
         5.0   1.0                            3
2.0      5.0   3.0                           17
         2.0   3.0                           10
         4.0   1.0                            7
               3.0                            6
         3.0   3.0                            

In [17]:
df[df['surgery'].isna()]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
132,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


    заполню surgery значением 1, так как pain в пропущенном surgery равняется 4, а 4 чаще встречается у лошадей у которых была операция. так же temperature_of_extremities = 3 холодные конечности, часто связаны с операциейей. так же чаще встречается у тех лошадей у которых была операцция.
    
    Пропуск в outcome заполнен значением 2, так как для условий pain=4 и temperature_of_extremities=3 это значение встречается чаще (2 раза), чем 3.0 (1 раз). Решение основано на анализе распределения данных который я сдела на пару ячеек выше.
    
    
    получается мои теории здесь:
        1) если у лошади сильная боль и у нее холодные конечности, то она перенесла операцию.
        данные выше показали что pain 4 and temperature_of_extremities 3 чаще встречаются у лошадей перенесших операцию
        
        2) так же для outcome моя теория была что если лошадь с pain 4 and temperature_of_extremities 3, то она умерла, я просто посмотрел, что чаще встречается. умерла сама или сделали эвтаназию.
        
    

In [18]:
df.loc[df['surgery'].isna(), 'surgery'] = 1
df.loc[df['outcome'].isna(), 'outcome'] = 2


In [19]:
df[df['outcome'].isna()]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome


In [20]:
df.isna().sum()

surgery                        0
age                            0
rectal_temperature            60
pulse                         24
respiratory_rate              58
temperature_of_extremities    56
pain                          55
outcome                        0
dtype: int64

    непрерывные переменные continuos_var = ['rectal_temperature', 'pulse', 'respiratory_rate']
    я решил выше заполнить медианой, потому что среднее менялось когда смотрел с выбросами и нет, мода здесь не подходит, а медиана практически не менялась или менялась не значительно

In [21]:
median_pulse = df['pulse'].median()
median_rectal_temperature = df['rectal_temperature'].median()
median_respiratory_rate = df['respiratory_rate'].median()
median_pulse, median_rectal_temperature, median_respiratory_rate

(64.0, 38.2, 24.5)

In [22]:
df['pulse'].fillna(median_pulse, inplace=True)
df['rectal_temperature'].fillna(median_rectal_temperature, inplace=True)
df['respiratory_rate'].fillna(median_respiratory_rate, inplace=True)

In [23]:
df.isna().sum()

surgery                        0
age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities    56
pain                          55
outcome                        0
dtype: int64

In [48]:
print("surgery*****************pain")
print(df.groupby('surgery')['pain'].value_counts())
print("outcome*****************pain")
print(df.groupby('outcome')['pain'].value_counts())
print("temperature_of_extremities*****************pain")
print(df.groupby('temperature_of_extremities')['pain'].value_counts())



surgery*****************pain
surgery  pain
1.0      3.0     39
         2.0     34
         5.0     33
         4.0     29
         1.0      9
2.0      1.0     29
         3.0     28
         2.0     25
         4.0     10
         5.0      9
Name: count, dtype: int64
outcome*****************pain
outcome  pain
1.0      3.0     54
         1.0     35
         2.0     33
         5.0     12
         4.0     10
2.0      5.0     24
         4.0     18
         2.0     14
         3.0      8
         1.0      2
3.0      2.0     12
         4.0     11
         5.0      6
         3.0      5
         1.0      1
Name: count, dtype: int64
temperature_of_extremities*****************pain
temperature_of_extremities  pain
1.0                         3.0     26
                            1.0     18
                            2.0     16
                            4.0      7
                            5.0      6
2.0                         1.0     11
                            3.0      9
        

In [58]:

# создам группы для rectal_temperature что бы потом посмотреть как влияет это на температуру в конечностях
def categorize_temp(temp):
    if temp < 37.8:
        return 'Low'
    elif 37.8 <= temp <= 38.5:
        return 'Normal'
    else:
        return 'High'

df['temp_group'] = df['rectal_temperature'].apply(categorize_temp)

In [61]:
df.groupby('temp_group')['temperature_of_extremities'].value_counts()


temp_group  temperature_of_extremities
High        3.0                           21
            1.0                           12
            2.0                            9
            4.0                            7
Low         3.0                           23
            1.0                           12
            2.0                            6
            4.0                            3
Normal      3.0                           65
            1.0                           54
            4.0                           17
            2.0                           15
Name: count, dtype: int64

    я не увидел закономерности между прохладными или нет конечностями и температурой.

In [62]:
df.groupby('surgery')['temperature_of_extremities'].value_counts()

surgery  temperature_of_extremities
1.0      3.0                           67
         1.0                           43
         4.0                           20
         2.0                           11
2.0      3.0                           42
         1.0                           35
         2.0                           19
         4.0                            7
Name: count, dtype: int64

In [65]:
# если лошадь жива, то температура конечностей нормальная(глупо звучит, но все же)
df.groupby('outcome')['temperature_of_extremities'].value_counts()

outcome  temperature_of_extremities
1.0      1.0                           63
         3.0                           46
         2.0                           26
         4.0                           11
2.0      3.0                           42
         1.0                           11
         4.0                            9
         2.0                            3
3.0      3.0                           21
         4.0                            7
         1.0                            4
         2.0                            1
Name: count, dtype: int64

In [78]:
df.groupby(['outcome', 'surgery'])['temperature_of_extremities'].value_counts()

outcome  surgery  temperature_of_extremities
1.0      1.0      1.0                           30
                  3.0                           30
                  2.0                            8
                  4.0                            7
         2.0      1.0                           33
                  2.0                           18
                  3.0                           16
                  4.0                            4
2.0      1.0      3.0                           28
                  1.0                           10
                  4.0                            8
                  2.0                            2
         2.0      3.0                           14
                  1.0                            1
                  2.0                            1
                  4.0                            1
3.0      1.0      3.0                            9
                  4.0                            5
                  1.0                

    на основвании этих данных, заполню пропуски temperature_of_extremities так. если лошадь жива, то теммпература конечностей 1, а в остальных случаях 3

In [80]:
def fill_temp_ext(row):
    if pd.isna(row['temperature_of_extremities']):  # Проверяем, является ли значение NaN
        if row['outcome'] == 1:
            row['temperature_of_extremities'] = 1.0
        elif row['outcome'] in [2,3]:
            row['temperature_of_extremities'] = 3.0
    return row


In [83]:
df = df.apply(fill_temp_ext, axis=1)


In [84]:
df['temperature_of_extremities'].value_counts()

temperature_of_extremities
3.0    133
1.0    110
2.0     30
4.0     27
Name: count, dtype: int64

In [104]:
# теория такая. если произвели эвтаназию или лошадб умерла, то боль была адская
# и видно что если лошадь умерла то боль была 4 или 5
df.groupby('pain')['outcome'].agg(lambda x: x.mode()[0])

pain
1.0    1.0
2.0    1.0
3.0    1.0
4.0    2.0
5.0    2.0
Name: outcome, dtype: float64

In [105]:
df.groupby('outcome').agg({'pain': lambda x: x.mode()[0]})

,pain
outcome,
1.0,3.0
2.0,5.0
3.0,2.0


In [107]:
df.loc[(df['pain'].isna()) & (df['outcome'] == 2), 'pain'] = 5


In [108]:
df.isna().sum()

surgery                        0
age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities     0
pain                          43
outcome                        0
temp_group                     0
dtype: int64

In [109]:
df.pain.value_counts()

pain
3.0    67
2.0    59
5.0    54
4.0    39
1.0    38
Name: count, dtype: int64

In [110]:
# теория вот такая. если операции не было. то боль низкая
df.groupby('pain')['surgery'].agg(lambda x: x.mode()[0])


pain
1.0    2.0
2.0    1.0
3.0    1.0
4.0    1.0
5.0    1.0
Name: surgery, dtype: float64

In [113]:
df.loc[(df['pain'].isna()) & (df['surgery'] == 2), 'pain'] = 1


In [116]:
pain_mode = df['pain'].mode()[0]
df['pain'].fillna(pain_mode, inplace=True)


In [117]:
df.pain.value_counts()

pain
3.0    94
2.0    59
5.0    54
1.0    54
4.0    39
Name: count, dtype: int64

In [118]:
df.isna().sum()

surgery                       0
age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
temp_group                    0
dtype: int64

In [120]:

df.columns

Index(['surgery', 'age', 'rectal_temperature', 'pulse', 'respiratory_rate',
       'temperature_of_extremities', 'pain', 'outcome', 'temp_group'],
      dtype='object')

In [122]:
clean_df = df[['surgery', 'age', 'rectal_temperature', 'pulse', 'respiratory_rate',
       'temperature_of_extremities', 'pain', 'outcome']].copy()

In [125]:
clean_df.isna().sum()

surgery                       0
age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64

In [126]:
clean_df.to_csv('horse_clean.csv')

# если заполять все модой надо было
    pain_mode = df['pain'].mode()[0]  
    df['pain'].fillna(pain_mode, inplace=True)  
    
    temp_mode = df['temperature_of_extremities'].mode()[0]  
    df['temperature_of_extremities'].fillna(temp_mode, inplace=True)